In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from io import BytesIO
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import datetime
from selenium.webdriver.chrome.options import Options
import numpy as np
import random
from selenium.webdriver import ActionChains
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("../creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("stock data")
sheet = sheet.worksheet("crypto")
df = pd.DataFrame(sheet.get_all_values())
df.columns = df.iloc[0]
df = df.iloc[1:]
df = df.replace('N/A', np.nan)
df= df.dropna()

In [ ]:
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("../creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("stock data1")
sheet = sheet.worksheet("crypto")
df1 = pd.DataFrame(sheet.get_all_values())
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df1 = df1.replace('N/A', np.nan)
df1= df1.dropna()


In [ ]:
allData = df.append(df1)
allData

In [ ]:
allData = allData.reset_index()
allData

In [ ]:
allData = allData.drop(columns = ['index'])
allData

In [ ]:
# allData.to_csv('../crypto.csv',encoding='utf-8', index=False)

In [ ]:
df = allData

In [ ]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if "," in x:
        x = x.replace(',', '')
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000
        return 1000000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return float(x)

In [ ]:
df["Total Volume All Currencies (24Hr)"] = [value_to_float(x) for x in df["Total Volume All Currencies (24Hr)"]]
df["Volume in Currency (Since 0:00 UTC)"] = [value_to_float(x) for x in df["Volume in Currency (Since 0:00 UTC)"]]
df["Volume in Currency (24Hr)"] = [value_to_float(x) for x in df["Volume in Currency (24Hr)"]]
df['Circulating Supply'] = [value_to_float(x) for x in df['Circulating Supply']]
df['52 Week Range'] = [value_to_float(x) for x in df['52 Week Range']]

In [ ]:
df['Change'] = [s.replace(',', '').replace('+', '') for s in df['Change']]
df["Price (Intraday)"] = [s.replace(',', '') for s in df["Price (Intraday)"]]

In [ ]:
df['time'] = [datetime.datetime.strptime(s[4:], '%b %d %H:%M:%S %Y') for s in df['time']]

In [ ]:
rbh = ['BSV-USD','BTC-USD', 'BCH-USD','LTC-USD','ETH-USD']

In [ ]:
df = df.astype({'Change': float, "Price (Intraday)": float, "Total Volume All Currencies (24Hr)": float, "Volume in Currency (Since 0:00 UTC)": float, "Volume in Currency (24Hr)": float, 'Circulating Supply':float,'52 Week Range': float})


In [ ]:
df = df.drop(columns = '% Change')

In [ ]:
list(df['time'])[0].hour

In [ ]:
df['day of week'] = [t.weekday() for t in df['time']]
df

In [ ]:
df['hour of day'] = [t.hour for t in df['time']]
df
df = df[df['hour of day'] != 22]


In [ ]:
df['timedelta'] = [(datetime.datetime.now() - x).total_seconds() for x in df['time']]

In [ ]:
df['6 hour diff'] = df.groupby('Symbol')['Price (Intraday)'].diff() * -1

In [ ]:
df['% 6 hour diff'] = df['6 hour diff']/ (df['Price (Intraday)'] - df['6 hour diff']) * 100

In [ ]:
oneHotDOW = pd.get_dummies(df['day of week'],prefix='dow')
oHDOWcols = oneHotDOW.columns


In [ ]:
df[list(oHDOWcols)] = oneHotDOW

In [ ]:
df = df.drop(columns = 'day of week')
df

In [ ]:
oneHotHOD = pd.get_dummies(df['hour of day'],prefix='hod')
oHHODcols = oneHotHOD.columns

In [ ]:
df[list(oHHODcols )] = oneHotHOD

In [ ]:
df = df.drop(columns = 'hour of day')
df

In [ ]:
dfNoOH = df.iloc[:,:13]
dfNoOH

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)

In [ ]:
# def knn(data, k, test_size, train_size, guess):
    
#     data = data.drop(columns = 'time')
#     test = data.drop(columns = ['Name', 'Symbol'])
#     if not guess:
#         test = test.dropna()
#     X = test.drop(columns = ['% 6 hour diff', '6 hour diff'])
#     y = test['quartile'].astype(int)
#     scaler = StandardScaler() 
#     X = scaler.fit_transform(X)
    
#     X_train = X[test_size:test_size + train_size]
#     X_test = X[:test_size]

    
#     y_train = y[test_size:test_size + train_size]
#     y_test = y[:test_size]
    
#     model = KNeighborsClassifier(n_neighbors=k)
#     model.fit(X_train,y_train)
#     y_pred = model.predict(X_test)
#     return y_pred, y_test

In [ ]:
def knn(data, k, test_size, train_size, guess, q):
    
    bins = q
    qs = list(data['% 6 hour diff'].quantile(np.linspace(0,1, bins + 1)[1:bins]))
    qs
    quartile = []
    for x in data['% 6 hour diff']:
        for i in range(bins):
            if i == bins -1:
                quartile.append(i)
                break
            if x < qs[i]:
                quartile.append(i)
                break 
    data['quartile'] = quartile
    
    
    data = data.drop(columns = 'time')
    test = data.drop(columns = ['Name', 'Symbol'])
    if not guess:
        test = test.dropna()
        
    X = test.drop(columns = ['% 6 hour diff', '6 hour diff'])
    y = test['quartile'].astype(int)
    scaler = StandardScaler() 
    X = scaler.fit_transform(X)
    
    X_train = X[test_size:test_size + train_size]
    X_test = X[:test_size]

    
    y_train = y[test_size:test_size + train_size]
    y_test = y[:test_size]
    
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return y_pred, y_test, qs

In [ ]:
rbh = ['ETH-USD','BTC-USD','LTC-USD','BCH-USD', 'BSV-USD']

In [ ]:
allResults = pd.DataFrame()
for q in range(2, 15):
    p, t, qs = knn(df, 10, 30, 10000, True,q)
    results = df.iloc[:15]
    results['bins'] = [q] * 15
    results['guess'] = p[:15]
    maxProjs = []
    maxVals = []
    for i in range(len(results['guess'])):
        guess = results['guess'][i]
        currVal = results['Price (Intraday)'][i]
        if guess == 0:
            maxP = 'x < ' + str(qs[guess]) 
            maxVal = 'x < ' + str(qs[guess]/100 * currVal + currVal) 
        elif guess == q - 1:
            maxP =  str(qs[guess -1]) + ' < x'
            maxVal= str(qs[guess -1]/100 * currVal + currVal) + ' < x'
        else:
            maxP = str(qs[guess - 1]) + ' < ' + ' x ' + ' < ' + str(qs[guess])
            maxVal = str(qs[guess - 1]/100 * currVal + currVal) + ' < ' + ' x ' + ' < ' + str(qs[guess]/ 100 * currVal + currVal)
        maxVals.append(maxVal)
        maxProjs.append(maxP)
    results['max value'] = maxVals
    results['max %'] = maxProjs
    allResults = allResults.append(results[['Symbol','Name','Price (Intraday)','bins', 'guess', 'max %','max value']])
allResults

In [ ]:
keep = []
for x in allResults['Symbol']:
    if x in rbh:
        keep.append(True)
    else:
        keep.append(False)

In [ ]:
allResults = allResults[keep]

In [ ]:
allResults = allResults.sort_values(['Symbol', 'bins'])
allResultsOneHot = allResults
allResultsOneHot 

In [ ]:
allResults = pd.DataFrame()
for q in range(2, 15):
    p, t, qs = knn(dfNoOH, 10, 30, 10000, True,q)
    results = df.iloc[:15]
    results['bins'] = [q] * 15
    results['guess'] = p[:15]
    maxProjs = []
    maxVals = []
    for i in range(len(results['guess'])):
        guess = results['guess'][i]
        currVal = results['Price (Intraday)'][i]
        if guess == 0:
            maxP = 'x < ' + str(qs[guess]) 
            maxVal = 'x < ' + str(qs[guess]/100 * currVal + currVal) 
        elif guess == q - 1:
            maxP =  str(qs[guess -1]) + ' < x'
            maxVal= str(qs[guess -1]/100 * currVal + currVal) + ' < x'
        else:
            maxP = str(qs[guess - 1]) + ' < ' + ' x ' + ' < ' + str(qs[guess])
            maxVal = str(qs[guess - 1]/100 * currVal + currVal) + ' < ' + ' x ' + ' < ' + str(qs[guess]/ 100 * currVal + currVal)
        maxVals.append(maxVal)
        maxProjs.append(maxP)
    results['max value'] = maxVals
    results['max %'] = maxProjs
    allResults = allResults.append(results[['Symbol','Name','Price (Intraday)','bins', 'guess', 'max %','max value']])
allResults

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)

In [ ]:
keep = []
for x in allResults['Symbol']:
    if x in rbh:
        keep.append(True)
    else:
        keep.append(False)

In [ ]:
allResults = allResults[keep]

In [ ]:
allResults = allResults.sort_values(['Symbol', 'bins'])
allResults

In [ ]:
# #TESTER

scores = pd.DataFrame(columns=['Symbol', 'Current Price', "bins", "average accuracy",'guess',"max %",'max value','true positive rate','pred > true','pred < true','pred < q0'])

for symbol in rbh:
    data = df[df['Symbol'] == symbol]
    currVal = list(data['Price (Intraday)'])[0]
    print(symbol)

    for x in range(2,18):
        bins = x
#         qs = list(data['% 6 hour diff'].quantile(np.linspace(0,1, bins + 1)[1:bins]))
#         quartile = []
#         for x in data['% 6 hour diff']:
#             for i in range(bins):
#                 if i == bins -1:
#                     quartile.append(i)
#                     break
#                 if x < qs[i]:
#                     quartile.append(i)
#                     break 
#         data['quartile'] = quartile

        samples = 800
        if symbol == 'BSV-USD':
            samples = 120
        train_size = 248
        test_size = 2
        # for k in range(1,6):


        k = 10
        p1 = []
        t1 = []
        for y in range(1, samples):
            test = data.iloc[y:]
            p, t, bb = knn(test, k, test_size, train_size, False, x)
            p1.append(p[0])
            t1.append(t.iloc[0])
        avgr2 = metrics.accuracy_score(t1,p1)
        predss = pd.DataFrame({'y_pred':p1, 'y_test':t1, '% 6 hour diff' : list(data['% 6 hour diff'])[1:len(p1)+1]})
        tp = 0
        pGt = 0
        pLt = 0
        pG0 = 0
        

    #     tp = tp/len(predss['y_pred']) * 100
    #     pGt = pGt/len(predss['y_pred']) * 100
    #     pLt = pLt/len(predss['y_pred']) * 100
    #     pG0 = pG0/len(predss['y_pred']) * 100



        p, t, qs = knn(data, k, test_size, train_size, True, x)
        for i in range(len(qs)):
            if qs[i] > 0:
                qInd = i
                break
        guess = list(p)[0]
        predsProj = pd.DataFrame({'y_pred':p1, 'y_test':t1})
        


        predss = predss[predss['y_pred'] == guess]
        for i in range(len(predss['y_pred'])):
            pred = list(predss['y_pred'])[i]
            test = list(predss['y_test'])[i]
            if pred == test:
                tp += 1
            elif pred > test:
                pGt += 1
            elif pred < test:
                pLt += 1
                if pred <= qInd:
                    pG0 += 1
        if guess == 0:
            maxP = 'x < ' + str(qs[guess]) 
        elif guess == bins - 1:
            maxP =  str(qs[guess -1]) + ' < x'
        else:
            maxP = str(qs[guess - 1]) + ' < ' + ' x ' + ' < ' + str(qs[guess])
        if guess == 0:
            maxVal = 'x < ' + str(qs[guess]/100 * currVal + currVal) 
        elif guess == bins - 1:
            maxVal= str(qs[guess -1]/100 * currVal + currVal) + ' < x'
        else:
            maxVal = str(qs[guess - 1]/100 * currVal + currVal) + ' < ' + ' x ' + ' < ' + str(qs[guess]/ 100 * currVal + currVal)
        scores = scores.append({'Symbol': symbol,
                                'Current Price': currVal,
                            'bins': bins,
                            "average accuracy": avgr2,
                                'guess' : guess,
                               "max %": maxP,
                               'max value' : maxVal,
                               'true positive rate': tp,
                               'pred > true': pGt,
                               'pred < true': pLt,
                               'pred < q0': pG0,
                               'q0': qs[qInd]}, ignore_index=True)
scores

In [ ]:
allResults

In [ ]:
allResultsOneHot

In [ ]:
scores

In [ ]:
data = df[df['Symbol'] == 'ETH-USD']
x = 3
p1 = []
t1 = []
for y in range(0, samples):
    test = data.iloc[y:]
    p, t = knn(test, k, test_size, train_size, False, x)
    p1.append(p[0])
    t1.append(t.iloc[0])
avgr2 = metrics.accuracy_score(t1,p1)
predss = pd.DataFrame({'y_pred':p1, 'y_test':t1, '% 6 hour diff' : list(data['% 6 hour diff'])[1:len(p1)+1]})
y_true = predss['y_test']
y_pred = predss['y_pred']
unique_label = list(set(y_true))
pd.DataFrame(
    metrics.confusion_matrix(y_true, y_pred, labels=unique_label), 
    index=['true:{:}'.format(x) for x in unique_label], 
    columns=['pred:{:}'.format(x) for x in unique_label]
)

In [ ]:
print(avgr2)
predss.head(40)

In [ ]:
testSize = 30
trainSize = 10000
k = 10
q = 10

In [ ]:
p, t, qs= knn(dfNoOH, k,testSize, trainSize, False,q)
metrics.accuracy_score(p,t)

In [ ]:
p, t, qs= knn(df,  k,testSize, trainSize, False,q)
metrics.accuracy_score(p,t)

In [ ]:
bins = 17
qs = list(eth['% 6 hour diff'].quantile(np.linspace(0,1, bins + 1)[1:bins]))
qs
quartile = []
for x in eth['% 6 hour diff']:
    for i in range(bins):
        if i == bins -1:
            quartile.append(i)
            break
        if x < qs[i]:
            quartile.append(i)
            break 
eth['quartile'] = quartile

In [ ]:
#LONG TERM
minR2 = 0
ind = 0
xx = 0
data = eth
tests = pd.DataFrame(columns=["test size", "train size", "k neighbors", "average accuracy",])
# for k in range(1,12):
k = 1
for x in range(248, 250):
    print(x)
    for i in range(1, 3):
        avgr2 = 0
        p1 = []
        t1 = []
        for y in range(1, 800):
            test = data.iloc[y:]
            p, t = knn(test, k, i, x, False, bins)
            p1.append(p[0])
            t1.append(t.iloc[0])
        avgr2 = metrics.accuracy_score(t1,p1)
        tests = tests.append({"test size": i,
                        "train size": x,
                        "k neighbors": k,
                        "average accuracy": avgr2}, ignore_index=True)
        if minR2 < avgr2 and avgr2 < 1:
            minR2 = avgr2
            ind = i
            xx = x
            ts = t1
            ps = p1          
ind

In [ ]:
tests = tests.sort_values('average accuracy', ascending = False).head(40)
guesses = []
for i in range(len(list(tests['train size']))):
    p, t = knn(eth, int(list(tests["k neighbors"])[i]), int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True, bins)
    predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
    guess = list(predsProj['y_pred'])[0]
    guesses.append(guess)
tests['guess'] = guesses  
tests

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)


In [ ]:
predss = pd.DataFrame({'y_pred':ps, 'y_test':ts, '% 6 hour diff' : list(eth['% 6 hour diff'])[1:len(ps)+1]})
display(predss[predss['y_pred'] == list(tests['guess'])[0]])

In [ ]:
qs[2]

In [ ]:
y_true = predss['y_test']
y_pred = predss['y_pred']
unique_label = list(set(y_true))
pd.DataFrame(
    metrics.confusion_matrix(y_true, y_pred, labels=unique_label), 
    index=['true:{:}'.format(x) for x in unique_label], 
    columns=['pred:{:}'.format(x) for x in unique_label]
)

In [ ]:
metrics.confusion_matrix(predss['y_test'], predss['y_pred'])

In [ ]:
#GUESS FOR 100 of the last values, train (115-150), test (1-3), 5 neighbors 
#and practically
#GUESS FOR 100 of the last values, train (1-150), test (2-100), 5 neighbors

In [ ]:
# tests = tests.sort_values('average accuracy', ascending = False).head(40)
# guesses = []
# for i in range(40):
#     p, t = knn(eth, int(list(tests["k neighbors"])[i]), int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True)
#     predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
#     guess = list(predsProj['y_pred'])[0]
#     guesses.append(guess)
# tests['guess'] = guesses  
# tests

In [ ]:
#SHORT TERM
minR2 = 0
ind = 0
xx = 0
data = eth
tests = pd.DataFrame(columns=["test size", "train size", "k neighbors", "average accuracy",])

k =5
# for k in range(1,12):
for x in range(70, 80):
    print(x)
    for i in range(22, 30):
        avgr2 = 0
        p1 = []
        t1 = []
        for y in range(1, 500):
            test = data.iloc[y:]
            p, t = knn(test, k, i, x, False)
            p1.append(p[0])
            t1.append(t.iloc[0])
        avgr2 = metrics.accuracy_score(t1,p1)
        tests = tests.append({"test size": i,
                        "train size": x,
                        "k neighbors": k,
                        "average accuracy": avgr2}, ignore_index=True)
        if minR2 < avgr2 and avgr2 < 1:
            minR2 = avgr2
            ind = i
            xx = x
            ts = t1
            ps = p1          
ind

In [ ]:
qs[2]

In [ ]:
#GUESS FOR 15 of the last values, train (65 - 79), test (22-30), 5 neighbors

In [ ]:
tests = tests.sort_values('average accuracy', ascending = False).head(40)
guesses = []
for i in range(40):
    p, t = knn(eth, int(list(tests["k neighbors"])[i]), int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True)
    predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
    guess = list(predsProj['y_pred'])[0]
    guesses.append(guess)
tests['guess'] = guesses  
tests

In [ ]:
tests['guess'].value_counts().index[0]

In [ ]:
predss = pd.DataFrame({'y_pred':ps, 'y_test':ts})
predss

In [ ]:
qs = list(eth['% 6 hour diff'].quantile([.25,0.5,.75]))
qs

In [ ]:
len(quartile)

In [ ]:
quartile = []
for x in eth['% 6 hour diff']:
    if x < qs[0]:
        quartile.append(0)
    elif x < qs[1]:
        quartile.append(1)
    elif x < qs[2]:
        quartile.append(2)
    else:
        quartile.append(3)
        

In [ ]:
eth['quartile'] = quartile

In [ ]:
#LONG TERM
minR2 = 0
ind = 0
xx = 0
data = eth
tests = pd.DataFrame(columns=["test size", "train size", "k neighbors", "average accuracy",])
# for k in range(1,12):
k = 5
for x in range(110, 150):
    print(x)
    for i in range(1, 3):
        avgr2 = 0
        p1 = []
        t1 = []
        for y in range(1, 100):
            test = data.iloc[y:]
            p, t = knn(test, 5, i, x, False)
            p1.append(p[0])
            t1.append(t.iloc[0])
        avgr2 = metrics.accuracy_score(t1,p1)
        tests = tests.append({"test size": i,
                        "train size": x,
                        "k neighbors": k,
                        "average accuracy": avgr2}, ignore_index=True)
        if minR2 < avgr2 and avgr2 < 1:
            minR2 = avgr2
            ind = i
            xx = x
            ts = t1
            ps = p1          
ind

In [ ]:
tests = tests.sort_values('average accuracy', ascending = False).head(40)
guesses = []
for i in range(40):
    p, t = knn(eth, int(list(tests["k neighbors"])[i]), int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True)
    predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
    guess = list(predsProj['y_pred'])[0]
    guesses.append(guess)
tests['guess'] = guesses  
tests

In [ ]:
qs[2]

In [ ]:
predss = pd.DataFrame({'y_pred':ps, 'y_test':ts})
predss.head(20)

In [ ]:
#SHORT TERM
minR2 = 0
ind = 0
xx = 0
data = eth
tests = pd.DataFrame(columns=["test size", "train size", "k neighbors", "average accuracy",])

k =5
# for k in range(1,12):
for x in range(50, 80):
    print(x)
    for i in range(22, 30):
        avgr2 = 0
        p1 = []
        t1 = []
        for y in range(1, 20):
            test = data.iloc[y:]
            p, t = knn(test, k, i, x, False)
            p1.append(p[0])
            t1.append(t.iloc[0])
        avgr2 = metrics.accuracy_score(t1,p1)
        tests = tests.append({"test size": i,
                        "train size": x,
                        "k neighbors": k,
                        "average accuracy": avgr2}, ignore_index=True)
        if minR2 < avgr2 and avgr2 < 1:
            minR2 = avgr2
            ind = i
            xx = x
            ts = t1
            ps = p1          
ind

In [ ]:
#GUESS FOR 15 of the last values, train (65 - 79), test (22-30), 5 neighbors

In [ ]:
tests = tests.sort_values('average accuracy', ascending = False).head(40)
guesses = []
for i in range(40):
    p, t = knn(eth, int(list(tests["k neighbors"])[i]), int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True)
    predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
    guess = list(predsProj['y_pred'])[0]
    guesses.append(guess)
tests['guess'] = guesses  
tests

In [ ]:
tests['guess'].value_counts().index[0]

In [ ]:
predss = pd.DataFrame({'y_pred':ps, 'y_test':ts})
predss

In [ ]:
aaa

In [ ]:
tests = tests.sort_values('average accuracy', ascending = False).head(40)
guesses = []
for i in range(40):
    p, t = knn(eth, int(list(tests['test size'])[i]), int(list(tests['train size'])[i]), True)
    predsProj = pd.DataFrame({'y_pred':p, 'y_test':t})
    guess = list(predsProj['y_pred'])[0]
    guesses.append(guess)
tests['guess'] = guesses  
tests

In [ ]:
bch = df[df['Symbol'] == 'BCH-USD']
ltc = df[df['Symbol'] == 'LTC-USD']
zec = df[df['Symbol'] == 'ZEC-USD']
xrp = df[df['Symbol'] == 'XRP-USD']
xmr = df[df['Symbol'] == 'XMR-USD']
neo = df[df['Symbol'] == 'NEO-USD']
dash = df[df['Symbol'] == 'DASH-USD']
xlm = df[df['Symbol'] == 'XLM-USD']
bsv = df[df['Symbol'] == 'BSV-USD']
eth = df[df['Symbol'] == 'ETH-USD']
btc = df[df['Symbol'] == 'BTC-USD']

In [ ]:
result = pd.DataFrame(columns = ['Symbol', 'cutoff', 'accuracy', 'over cutoff'])
for frame in frames:
    sym = list(frame['Symbol'])[0]
    for i in np.arange(0,10, .05):
        try:
            p, t, per, act = logRegressionScaled(frame, i)
            acc = metrics.accuracy_score(t, p)
            act = act[0]
            result = result.append({"Symbol" : sym,
                                            'cutoff': i, 
                                            "accuracy": acc, 
                                            "over cutoff": act}, ignore_index=True)
        except:
            continue

In [ ]:
result[(result['over cutoff']) & (result['Symbol'] == 'ETH-USD')].sort_values('cutoff', ascending = False).head(40)